In [2]:
import mlflow

# Connect to remote MLflow server
mlflow.set_tracking_uri("http://localhost:5001")
mlflow.set_experiment("Hyperparameter Tuning")

2025/11/26 19:14:26 INFO mlflow.tracking.fluent: Experiment with name 'Hyperparameter Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow/4', creation_time=1764180866635, experiment_id='4', last_update_time=1764180866635, lifecycle_stage='active', name='Hyperparameter Tuning', tags={}>

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

X, y = fetch_california_housing(return_X_y=True)
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0)

In [4]:
import mlflow
import optuna
import sklearn
from mlflow.models.signature import infer_signature


def objective(trial):
    # Setting nested=True will create a child run under the parent run.
    with mlflow.start_run(nested=True, run_name=f"trial_{trial.number}") as child_run:
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32)
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 50, 300, step=10)
        rf_max_features = trial.suggest_float("rf_max_features", 0.2, 1.0)
        params = {
            "max_depth": rf_max_depth,
            "n_estimators": rf_n_estimators,
            "max_features": rf_max_features,
        }
        # Log current trial's parameters
        mlflow.log_params(params)

        regressor_obj = sklearn.ensemble.RandomForestRegressor(**params)
        regressor_obj.fit(X_train, y_train)

        y_pred = regressor_obj.predict(X_val)
        error = sklearn.metrics.mean_squared_error(y_val, y_pred)
        # Log current trial's error metric
        mlflow.log_metrics({"error": error})

        signature = infer_signature(X_train, regressor_obj.predict(X_train))

        # Log the model file
        mlflow.sklearn.log_model(regressor_obj, name="model", signature=signature, input_example=X_train[:1])
        # Make it easy to retrieve the best-performing child run later
        trial.set_user_attr("run_id", child_run.info.run_id)
        return error

/Users/hugoglez/Library/CloudStorage/OneDrive-IBM/Portfolio/fraud-detection/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Create a parent run that contains all child runs for different trials
with mlflow.start_run(run_name="study") as run:
    # Log the experiment settings
    n_trials = 30
    mlflow.log_param("n_trials", n_trials)

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials)

    # Log the best trial and its run ID
    mlflow.log_params(study.best_trial.params)
    mlflow.log_metrics({"best_error": study.best_value})
    if best_run_id := study.best_trial.user_attrs.get("run_id"):
        mlflow.log_param("best_child_run_id", best_run_id)

[I 2025-11-26 19:15:38,728] A new study created in memory with name: no-name-5b733f86-ad69-4698-883b-2457dc993dfd
[I 2025-11-26 19:15:44,574] Trial 0 finished with value: 0.3825415628255717 and parameters: {'rf_max_depth': 7, 'rf_n_estimators': 160, 'rf_max_features': 0.9821053726957094}. Best is trial 0 with value: 0.3825415628255717.


🏃 View run trial_0 at: http://localhost:5001/#/experiments/4/runs/7be81bc5999a43978ccbdca6d111e900
🧪 View experiment at: http://localhost:5001/#/experiments/4


[I 2025-11-26 19:16:01,091] Trial 1 finished with value: 0.2654340606078376 and parameters: {'rf_max_depth': 24, 'rf_n_estimators': 240, 'rf_max_features': 0.9284141493935935}. Best is trial 1 with value: 0.2654340606078376.


🏃 View run trial_1 at: http://localhost:5001/#/experiments/4/runs/77f4d7a2af9a41cc8fb2288fd10f8e1a
🧪 View experiment at: http://localhost:5001/#/experiments/4


[I 2025-11-26 19:16:03,417] Trial 2 finished with value: 0.6546732617013846 and parameters: {'rf_max_depth': 5, 'rf_n_estimators': 90, 'rf_max_features': 0.22489754132244216}. Best is trial 1 with value: 0.2654340606078376.


...

🏃 View run trial_27 at: http://localhost:5001/#/experiments/4/runs/6d5b255000c944f49274f16cf9bf6304
🧪 View experiment at: http://localhost:5001/#/experiments/4


[I 2025-11-26 19:20:01,091] Trial 28 finished with value: 0.24494978945728504 and parameters: {'rf_max_depth': 31, 'rf_n_estimators': 290, 'rf_max_features': 0.4162498615476927}. Best is trial 3 with value: 0.24031253707958997.


🏃 View run trial_28 at: http://localhost:5001/#/experiments/4/runs/08da9538808c4d6bad8e9bee35a53700
🧪 View experiment at: http://localhost:5001/#/experiments/4


[I 2025-11-26 19:20:17,908] Trial 29 finished with value: 0.2585684146402436 and parameters: {'rf_max_depth': 32, 'rf_n_estimators': 290, 'rf_max_features': 0.6479251800438999}. Best is trial 3 with value: 0.24031253707958997.


🏃 View run trial_29 at: http://localhost:5001/#/experiments/4/runs/aad3a25eafaf4a0b91b979a8d3c2cfea
🧪 View experiment at: http://localhost:5001/#/experiments/4
🏃 View run study at: http://localhost:5001/#/experiments/4/runs/8be05d5d5017499796f6377757a4cd9a
🧪 View experiment at: http://localhost:5001/#/experiments/4


In [ ]:
# Register the best model using the model URI
mlflow.register_model(
    model_uri="s3://mlflow/4/models/m-caa0e0fdf3bd4dee9c118ba877d0d8da/artifacts",
    name="housing-price-predictor",
)

Registered model 'housing-price-predictor' already exists. Creating a new version of this model...
2025/11/26 19:24:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: housing-price-predictor, version 2
Created version '2' of model 'housing-price-predictor'.


<ModelVersion: aliases=[], creation_timestamp=1764181491598, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1764181491598, metrics=None, model_id=None, name='housing-price-predictor', params=None, run_id='', run_link='', source='s3://mlflow/4/models/m-caa0e0fdf3bd4dee9c118ba877d0d8da/artifacts', status='READY', status_message=None, tags={}, user_id='', version='2'>